<a href="https://colab.research.google.com/github/majdsuleiman98/Multi-Tool-LangChain-Agent-with-Groq-Optimized-LLMs/blob/main/Multi_Tool_LangChain_Agent_with_Groq_Optimized_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
!pip install langchain langchain-community wikipedia langchain_groq python-dotenv

In [153]:
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.document_loaders import WikipediaLoader
from langchain import hub
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
load_dotenv()

True

In [154]:
def get_date_today(input : str=None) -> str:
    from datetime import date
    return f"\n{date.today()}\n"

get_today_date_tool = Tool(
    name="Get date today",
    func=get_date_today,
    description="Useful for getting today's date"
)

In [156]:
def parse_query(input_text: str) -> str:
    keywords = {
        "name": "first_name",
        "last name": "last_name",
        "email": "email",
        "age": "age",
        "role": "role",
        "address": "address"
    }
    for key, value in keywords.items():
        if key in input_text.lower():
            return value
    return None

def get_person_query(input_text: str) -> str:
    person_database = {
        "first_name": "jhon",
        "last_name": "wick",
        "email": "jhon@gmail.com",
        "age": "40",
        "role": "admin",
        "address": "New York"
    }
    query = parse_query(input_text)
    if query and query in person_database:
        return f"\n{query.capitalize()}: {person_database[query]}\n"
    return f"\nError: Could not understand the query or query not found in the database.\n"


get_person_query_tool = Tool(
    name="Get Person Query",
    func=get_person_query,
    description="Retrieve specific information about a person from the database. Provide a natural language query like 'What is my age?' or 'What is my address?'."
)

In [157]:
def search_wikipedia(query: str) -> str:
    page = WikipediaLoader(query=query, load_max_docs=1).load()
    if page:
      summary = page[0].metadata["summary"]
      return f"\n{summary}\n"
    else:
      return "\nInvalid query\n"

search_wikipedia_tool = Tool(
    name="Search Wikipedia",
    func=search_wikipedia,
    description="Search Wikipedia for information. Provide a query, and it will return a summary of the Wikipedia page if found."
)

In [158]:
def math_calculator(expression : str) -> str:
    try:
        result = eval(expression)
        return f"\n{result}\n"
    except Exception as e:
        return f"\nError: {e}\n"

math_calculator_tool = Tool(
    name="Math Calculator",
    func=math_calculator,
    description="Evaluates mathematical expressions. You can use basic operators such as +, -, *, % and /."
)

In [159]:
def summerize_text(text: str) -> str:
    try:
      from transformers import pipeline
      summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
      summary = summarizer(text, do_sample=False)[0]['summary_text']
      return f"\n{summary}\n"
    except Exception as e:
        return "No text to summerize"

summerize_text_tool = Tool(
    name="Summerize Text",
    func=summerize_text,
    description="Useful for getting summarized long text"
)

In [160]:
def python_code_executor(code: str) -> str:
    try:
        exec_locals = {}
        exec(code, {}, exec_locals)
        return str(exec_locals)
    except Exception as e:
        return f"Error: {str(e)}"

python_code_executor_tool = Tool(
    name="Python Code Executor",
    func=python_code_executor,
    description="Executes Python code and returns the result."
)

In [161]:
ReActPrompt = hub.pull("hwchase17/react")
print(ReActPrompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [162]:
tools = [
    math_calculator_tool,
    get_today_date_tool,
    get_person_query_tool,
    search_wikipedia_tool,
    python_code_executor_tool,
    summerize_text_tool,
]

In [163]:
def Create_Agent(model,tools=tools):
  GROQ_API_KEY = os.getenv("GROQ_API_KEY")
  model = ChatGroq(model_name=model,groq_api_key=GROQ_API_KEY,temperature=0)
  ReActAgent = create_react_agent(model, tools=tools, prompt=ReActPrompt)
  ReActAgent_Executor = AgentExecutor(
    agent=ReActAgent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
  )
  return ReActAgent_Executor

In [121]:
ReActAgent_Executor = Create_Agent("llama3-70b-8192")

In [165]:
while True:
  query = input("Enter you query : ")
  if query == "stop":
    break
  ReActAgent_Executor.invoke({"input":query})

Enter you query : When was Lionel Messi born?


> Entering new AgentExecutor chain...
Thought: I need to search for information about Lionel Messi's birthdate.

Action: Search Wikipedia
Action Input: Lionel Messi
Lionel Andrés Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times being named the world's best player by FIFA. He is the most decorated player in the history of professional football having won 45 team trophies, including twelve Big Five league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Gold

In [166]:
ReActAgent_Executor = Create_Agent("mixtral-8x7b-32768")

In [167]:
while True:
  query = input("Enter you query : ")
  if query == "stop":
    break
  ReActAgent_Executor.invoke({"input":query})

Enter you query : When was Lionel Messi born?


> Entering new AgentExecutor chain...
To answer this question, I need to look up specific information about a person from the database. I can use the Get Person Query tool for this.

Action: Get Person Query
Action Input: "When was Lionel Messi born?"
Error: Could not understand the query or query not found in the database.
It seems the Get Person Query tool was not able to understand or find the information in the database. I will try using a different query format.

Action: Get Person Query
Action Input: "Birthdate of Lionel Messi"
Error: Could not understand the query or query not found in the database.
It appears that the Get Person Query tool is not functioning as expected. I will try a different approach by searching Wikipedia for information about Lionel Messi.

Action: Search Wikipedia
Action Input: "Lionel Messi"
Lionel Andrés Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987), also known as Leo Messi, is 

<string>:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?


It seems like the Math Calculator function is having trouble interpreting the input as a mathematical expression. Let's try breaking down the input into separate calculations.

Action: Math Calculator
Action Input: 8%3
2
Now that I have calculated the result of 8%3, I can substitute it back into the original expression and continue solving it.

Action: Math Calculator
Action Input: 10*2/5*2
8.0
I now know the final answer.
Final Answer: The solution to the math problem 10*2/5(8%3) is 8.0.

> Finished chain.
Enter you query : What is date today?


> Entering new AgentExecutor chain...
I need to find out the current date.
Action: Get date today
Action Input: None
2024-11-18
I now know the final answer
Final Answer: The date today is 2024-11-18.

> Finished chain.
Enter you query : Can you excute this python code: print("result : ",2*2)


> Entering new AgentExecutor chain...
The user wants me to execute a Python code snippet. I can use the Python Code Executor tool for this task.
Action: